In [1]:
import transformers 
import tensorflow as tf 
#from transformers import TFMBartForConditionalGeneration , MBart50TokenizerFast
from transformers import TFAutoModelForSeq2SeqLM
from transformers import AutoTokenizer  #TFT5ForConditionalGeneration,T5Tokenizer

2025-06-11 20:40:54.732110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749674454.754218      80 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749674454.761080      80 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model_name = "Helsinki-NLP/opus-mt-en-fr"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    dropout=0.3, 
    attention_dropout=0.3
)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


tf_model.h5:   0%|          | 0.00/301M [00:00<?, ?B/s]

I0000 00:00:1749674470.786296      80 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749674470.786949      80 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-fr.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/kaggle/input/en-fr45/eng_-french.csv')
df.head()
from datasets import Dataset

raw_datasets = Dataset.from_pandas(df)

print(raw_datasets)


Dataset({
    features: ['English words/sentences', 'French words/sentences'],
    num_rows: 175621
})


In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("/kaggle/input/en-fr45/eng_-french.csv")

df.columns = ["en", "fr"]

def word_count(text):
    return len(text.strip().split())

df_filtered = df.copy()  

df_filtered["translation"] = df_filtered.apply(lambda row: {"en": row["en"], "fr": row["fr"]}, axis=1)

df_filtered = df_filtered.drop(columns=["en", "fr"])

# Split into train, validation, and test sets
train_df, temp_df = train_test_split(df_filtered, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Convert to HuggingFace DatasetDict
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['translation', '__index_level_0__'],
        num_rows: 140496
    })
    validation: Dataset({
        features: ['translation', '__index_level_0__'],
        num_rows: 17562
    })
    test: Dataset({
        features: ['translation', '__index_level_0__'],
        num_rows: 17563
    })
})


In [6]:
tokenizer("Hello, this is a sentence")

{'input_ids': [10537, 2, 67, 32, 15, 5776, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [7]:
max_input_len = 64
max_output_len = 64

def preprocess_function(examples):
    input = [ex['en'] for ex in examples['translation']]
    targets = [ex['fr'] for ex in examples['translation']]
    model_inputs = tokenizer(input , max_length=max_input_len, truncation = True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length = max_output_len,truncation = True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenizer.src_lang = "en"
tokenizer.tgt_lang = "fr"


sample = dataset['train'][:1]
preprocess_function(sample)

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[1123, 55, 340, 541, 240, 3698, 9689, 143, 54, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[34326, 21, 1148, 1027, 1604, 4059, 143, 30913, 99, 0]]}

In [9]:
tokenized_dataset = dataset.map(preprocess_function,batched=True)

Map:   0%|          | 0/140496 [00:00<?, ? examples/s]

Map:   0%|          | 0/17562 [00:00<?, ? examples/s]

Map:   0%|          | 0/17563 [00:00<?, ? examples/s]

In [10]:
epochs = 10
batch_size = 32

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    return_tensors="tf"  # correct string input
)

In [23]:
from transformers import DataCollatorForSeq2Seq

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    return_tensors="tf",
 #   pad_to_multiple_of = 16,
)


In [13]:
train_datset = model.prepare_tf_dataset(
    tokenized_dataset['train'],
    batch_size = batch_size,
    shuffle = True,
    collate_fn = data_collator,
)

In [14]:
validation_datset = model.prepare_tf_dataset(
    tokenized_dataset['validation'],
    batch_size = 8,
    shuffle = False,
    collate_fn = data_collator,
)

In [15]:
generation_datset = model.prepare_tf_dataset(
    tokenized_dataset['validation'],
    batch_size = 32,
    shuffle = False,
    collate_fn = generation_data_collator,
)

In [16]:
train_steps_per_epoch = len(train_datset)  # assuming train_datset is batched already
num_train_steps = train_steps_per_epoch * epochs
num_warmup_steps = int(0.1 * num_train_steps) 

In [17]:
from transformers import AdamWeightDecay
from transformers import create_optimizer
from tensorflow.keras.callbacks import EarlyStopping

#optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
#model.compile(optimizer = optimizer)

class SafeEarlyStopping(EarlyStopping):
    def _implements_train_batch_hooks(self): return False
    def _implements_test_batch_hooks(self): return False
    def _implements_predict_batch_hooks(self): return False


optimizer, lr_schedule = create_optimizer(
    init_lr=5e-5,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    weight_decay_rate=0.01
)

# Compile the model
model.compile(optimizer=optimizer)

# Early stopping to prevent overfitting
early_stopping = SafeEarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    train_datset,
    validation_data=validation_datset,
    epochs=epochs,
    callbacks=[early_stopping]
)

Epoch 1/10
4390/4390 [==============================] - 1174s 259ms/step - loss: 1.2158 - val_loss: 0.5673
Epoch 2/10
4390/4390 [==============================] - 1142s 260ms/step - loss: 0.8391 - val_loss: 0.5544
Epoch 3/10
4390/4390 [==============================] - 1134s 258ms/step - loss: 0.7245 - val_loss: 0.5447
Epoch 4/10
4390/4390 [==============================] - 1133s 258ms/step - loss: 0.6554 - val_loss: 0.5406
Epoch 5/10
4390/4390 [==============================] - 1132s 258ms/step - loss: 0.6082 - val_loss: 0.5352
Epoch 6/10
4390/4390 [==============================] - 1132s 258ms/step - loss: 0.5702 - val_loss: 0.5316
Epoch 7/10
4390/4390 [==============================] - 1128s 257ms/step - loss: 0.5413 - val_loss: 0.5251
Epoch 8/10
4390/4390 [==============================] - 1131s 258ms/step - loss: 0.5183 - val_loss: 0.5212
Epoch 9/10
4390/4390 [==============================] - 1135s 259ms/step - loss: 0.4980 - val_loss: 0.5186
Epoch 10/10
4390/4390 [==============

In [25]:
model.save_pretrained("my_model")
tokenizer.save_pretrained("my_model")

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}
  warnings.warn(


('my_model/tokenizer_config.json',
 'my_model/special_tokens_map.json',
 'my_model/vocab.json',
 'my_model/source.spm',
 'my_model/target.spm',
 'my_model/added_tokens.json')

In [27]:
model = TFAutoModelForSeq2SeqLM.from_pretrained("my_model")
tokenizer = AutoTokenizer.from_pretrained("my_model")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at my_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [28]:
model.save_pretrained("checkpoints/epoch4")
tokenizer.save_pretrained("checkpoints/epoch4")

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}
  warnings.warn(


('checkpoints/epoch4/tokenizer_config.json',
 'checkpoints/epoch4/special_tokens_map.json',
 'checkpoints/epoch4/vocab.json',
 'checkpoints/epoch4/source.spm',
 'checkpoints/epoch4/target.spm',
 'checkpoints/epoch4/added_tokens.json')

In [20]:
!pip install sacrebleu

In [ ]:
from transformers import DataCollatorForSeq2Seq

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    return_tensors="tf"  #  no pad_to_multiple_of here
)


In [29]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_dataset['validation'],
    shuffle=False,
    batch_size=32,
    collate_fn=generation_data_collator,
)

In [ ]:
for i in range(20):
    print(f"Input    : {dataset['test']['translation'][i]['en']}")
    print(f"Reference: {dataset['test']['translation'][i]['fr']}")
    print(f"Prediction: {all_predictions[i]}\n")

In [33]:
print(tokenized_dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 127085
    })
})


In [40]:
print(tokenized_dataset)  # This will show available keys like 'train', 'test', etc.


DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 127085
    })
})


In [ ]:
from transformers import TFMarianMTModel, MarianTokenizer
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset
import tensorflow as tf
import numpy as np
import evaluate

#Load model and tokenizer
model_path = "/kaggle/working/my_model"
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = TFMarianMTModel.from_pretrained(model_path)

split = "test" if "test" in dataset else "validation"

max_input_len = 64
max_output_len = 64

#Preprocessing function
def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_output_len, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    return_tensors="tf"
)

#Prepare tf dataset
eval_subset = tokenized_dataset["train"].select(range(500))

generation_dataset = model.prepare_tf_dataset(
    tokenized_dataset[split],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
)

#BLEU evaluation
bleu = evaluate.load("bleu")
predictions = []
references = []

for inputs, labels in generation_dataset:
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Generate
    generated_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_output_len,
        num_beams=1,
        do_sample=False
    )

    decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    predictions.extend(decoded_preds)
    references.extend([[ref] for ref in decoded_labels])  # BLEU expects list of list

results = bleu.compute(predictions=predictions, references=references)
print(f"BLEU Score: {results['bleu'] * 100:.2f}")


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at /kaggle/working/my_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


Map:   0%|          | 0/140496 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/17562 [00:00<?, ? examples/s]

Map:   0%|          | 0/17563 [00:00<?, ? examples/s]

BLEU Score: 53.72
